In [1]:
import torch
import torchvision.transforms as transforms               # include image preprocess tools
from torchvision.datasets import CIFAR100        # for loading images from Pytorch CIFAR
from torch.utils.data import DataLoader
import detectors
import timm
from src.temperature_scaling import ModelWithTemperature
from src.raps_hyp_opt import lambda_optimization_raps, k_reg_optimization

# check GPU status
print("Is CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load fine-tuned model
model = timm.create_model("resnet18_cifar100", pretrained=True)
model = model.to(device)

# reprocess the images from CIFAR
data_transform = transforms.Compose([
    transforms.ToTensor(),          # transfer to tensor
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))  # normalize
])
# load images from CIFAR10
dataset = CIFAR100(root="../../../data", train=False, download=True, transform=data_transform)

# tempreture scaling
temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, temperature = 4.85).to(device)
model.set_temperature(temp_scal_loader)

model.eval()

C:\Users\jiayang\anaconda3\envs\pytorch2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Is CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 3060 Ti
Files already downloaded and verified
Before temperature - NLL: 1.126, ECE: 0.050
Optimal temperature: 4.704
After temperature - NLL: 3.091, ECE: 0.662


ModelWithTemperature(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): Identity()
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-

# $\alpha$=0.1

In [2]:
lambda_values = [0, 1e-4, 1e-3, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 0.7, 1.0]
k_reg_values = [1, 2, 3, 5, 7, 9, 10]

# lambda optimization
print("Looking for optimal lambda...")
optimal_lambda = lambda_optimization_raps(model, dataset, lambda_values, k_reg = 5, device=device)
if optimal_lambda is None:
    print("No optimal lambda is found")
else:
    print(f"Optimal lambda is {optimal_lambda}\n")
    
    # k_reg optimization
    print("Looking for optimal k_reg...")
    optimal_k = k_reg_optimization(model, dataset, optimal_lambda, k_reg_values, device=device)
    
    if optimal_k is None:
        print("No optimal k_reg is found")
    else:
        print(f"Optimal k_reg is {optimal_k}")

Is CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 3060 Ti
Files already downloaded and verified
Before temperature - NLL: 1.131, ECE: 0.050
Optimal temperature: 4.704
After temperature - NLL: 3.096, ECE: 0.663
Looking for optimal lambda...
Optimal lambda is 0.5

Looking for optimal k_reg...
Optimal k_reg is 2


# $\alpha$=0.05

In [2]:
lambda_values = [0.1, 0.15, 0.2, 0.25, 0.3, 0.33, 0.35, 0.5]
k_reg_values = [1, 2, 3, 4, 5, 6, 7, 8]

# lambda optimization
print("Looking for optimal lambda...")
optimal_lambda = lambda_optimization_raps(model, dataset, lambda_values, k_reg = 4, device=device, alpha=0.05)
if optimal_lambda is None:
    print("No optimal lambda is found")
else:
    print(f"Optimal lambda is {optimal_lambda}\n")
    
    # k_reg optimization
    print("Looking for optimal k_reg...")
    optimal_k = k_reg_optimization(model, dataset, optimal_lambda, k_reg_values, device=device, alpha=0.05)
    
    if optimal_k is None:
        print("No optimal k_reg is found")
    else:
        print(f"Optimal k_reg is {optimal_k}")

Looking for optimal lambda...
Optimal lambda is 0.3

Looking for optimal k_reg...
Optimal k_reg is 3


In [3]:
lambda_values = [0.1, 0.15, 0.2, 0.25, 0.3, 0.33, 0.35, 0.5]
k_reg_values = [1, 2, 3, 4, 5, 6, 7, 8]

# lambda optimization
print("Looking for optimal lambda...")
optimal_lambda = lambda_optimization_raps(model, dataset, lambda_values, k_reg = 4, device=device, alpha=0.2)
if optimal_lambda is None:
    print("No optimal lambda is found")
else:
    print(f"Optimal lambda is {optimal_lambda}\n")
    
    # k_reg optimization
    print("Looking for optimal k_reg...")
    optimal_k = k_reg_optimization(model, dataset, optimal_lambda, k_reg_values, device=device, alpha=0.2)
    
    if optimal_k is None:
        print("No optimal k_reg is found")
    else:
        print(f"Optimal k_reg is {optimal_k}")

Looking for optimal lambda...
Optimal lambda is 0.3

Looking for optimal k_reg...
Optimal k_reg is 2
